<a href="https://colab.research.google.com/github/Tiabet/Complete_story/blob/master/KoGPT_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install transformers
!pip install accelerate
!pip install datasets
!pip install evaluate
!pip install rouge
!pip install konlpy

In [1]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, DataCollatorWithPadding, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset, load_dataset
from rouge import Rouge
import evaluate
from konlpy.tag import Okt
import os
import numpy as np

In [2]:
model_checkpoint = "skt/kogpt2-base-v2"

tokenizer = PreTrainedTokenizerFast.from_pretrained(model_checkpoint, bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>', padding_side='left')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [3]:
 model = GPT2LMHeadModel.from_pretrained(model_checkpoint)

# 모델 테스트

In [4]:
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the input data (input_ids) to the GPU
input_ids = input_ids.to(device)

# Move the model to the GPU (assuming your model is already initialized)
model.to(device)
gen_ids = model.generate(input_ids,
                           max_length=32,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)

gen_ids = gen_ids.to("cpu")
generated = tokenizer.decode(gen_ids[0])
generated

'근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.\n특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.\n또한 하루 30분'

정상적으로 작동하는 것 확인

# 데이터셋 전처리

In [73]:
dataset = load_dataset('drive/MyDrive', data_files={
    'train': 'nikluge-sc-2023-train.jsonl',
    'validation': 'nikluge-sc-2023-dev.jsonl',
    'test': 'nikluge-sc-2023-test.jsonl'
})

In [74]:
sentence1 = []
sentence3 = []

for dictionary in dataset['train']['input']:
    sentence1.append(dictionary["sentence1"])
    sentence3.append(dictionary["sentence3"])

In [75]:
for i in range(len(sentence1)):
  sentence_1 = "첫 번째 문장 : {}".format(sentence1[i])
  sentence1[i] = sentence_1
  sentence_3 = "세 번째 문장 : {} 두 번째 문장 :".format(sentence3[i])
  sentence3[i] = sentence_3

In [76]:
dataset['train'] = dataset['train'].add_column('sentence1', sentence1)
dataset['train'] = dataset['train'].add_column('sentence3', sentence3)
#dataset['train'] = dataset['train'].add_column('input_sentence',input_sentence)

In [77]:
sentence1_val = []
sentence3_val = []

for dictionary in dataset['validation']['input']:
    sentence1_val.append(dictionary["sentence1"])
    sentence3_val.append(dictionary["sentence3"])

for i in range(len(sentence1_val)):
  sentence_1 = "첫 번째 문장 : {}".format(sentence1_val[i])
  sentence1_val[i] = sentence_1
  sentence_3 = "세 번째 문장 : {} 두 번째 문장 :".format(sentence3_val[i])
  sentence3_val[i] = sentence_3

dataset['validation'] = dataset['validation'].add_column('sentence1', sentence1_val)
dataset['validation'] = dataset['validation'].add_column('sentence3', sentence3_val)
#dataset['validation'] = dataset['validation'].add_column('input_sentence', input_sentence_val)

In [78]:
sentence1_test = []
sentence3_test = []

for dictionary in dataset['test']['input']:
    sentence1_test.append(dictionary["sentence1"])
    sentence3_test.append(dictionary["sentence3"])

for i in range(len(sentence1_test)):
  sentence_1 = "첫 번째 문장 : {}".format(sentence1_test[i])
  sentence1_test[i] = sentence_1
  sentence_3 = "세 번째 문장 : {} 두 번째 문장 :".format(sentence3_test[i])
  sentence3_test[i] = sentence_3

dataset['test'] = dataset['test'].add_column('sentence1', sentence1_test)
dataset['test'] = dataset['test'].add_column('sentence3', sentence3_test)
#dataset['test'] = dataset['test'].add_column('input_sentence', input_sentence_test)

In [79]:
def tokenize_function(example):
    return tokenizer(example['sentence1'], example['sentence3'],example['output'],truncation=True, padding='max_length',max_length=64, return_tensors="pt")

In [80]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/120140 [00:00<?, ? examples/s]

Map:   0%|          | 0/15017 [00:00<?, ? examples/s]

Map:   0%|          | 0/15018 [00:00<?, ? examples/s]

In [81]:
tokenized_datasets = tokenized_datasets.remove_columns(dataset["train"].column_names)

In [82]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 120140
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 15017
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 15018
    })
})

In [83]:
tokenizer.decode(tokenized_datasets['validation']['input_ids'][1], skip_special_tokens= False)

'<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> 첫 번째 문장 : 그는 도박을 하기 위해 부모님 돈을 가져다 썼다. 세 번째 문장 : 결국 부모님 돈은 탕진되었고 그는 엄청난 원망을 받았다. 두 번째 문장 :'

In [84]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #동적 패딩, attention_mask, input_ids 등의 길이를 똑같이 맞춰줌

# 평가 지표 작성 및 모델 파인튜닝

평가 지표에서 차이를 보이고 있는 부분들이 있어서, 두 평가 지표를 작성해보고자 한다.

In [ ]:
def evaluate(eval_preds):
    references, predictions = eval_preds

    mecab = Mecab()

    for idx, sentence_array in enumerate(references):
        tokenized = []
        for mor in mecab.morphs(sentence_array[0]):
            tokenized.append(mor)
        references[idx] = tokenized

    for idx, s in enumerate(predictions):
        tokenized = []
        for mor in mecab.morphs(s):
            tokenized.append(mor)
        predictions[idx] = tokenized

    for idx,sentence in enumerate(predictions):
      together = " ".join(sentence)
      predictions[idx] = together
    for idx,sentence in enumerate(references):
      together = " ".join(sentence)
      references[idx] = together

    # Calculate ROUGE-1 score
    rouge = Rouge()
    results = rouge.get_scores(predictions, references, avg=True)
    rouge_1_score = results['rouge-1']['f']

    #Calculate Bleu Score
    google_bleu = evaluate.load("google_bleu")
    results = google_bleu.compute(predictions=predictions, references=references)
    bleu_score = results['google_bleu']

    #Calculate BERTScore
    bertscore = evaluate.load("bertscore")
    results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
    bert_score_f1 = sum(results['f1'])/len(predictions)

    # Calculate the mean of the three scores
    mean_score = (rouge_1_score + bleu_score + bert_score_f1) / 3.0

    return mean_score

In [85]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    okt = Okt()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = [" ".join(okt.morphs(pred.strip())) for pred in decoded_preds]
    decoded_labels = [" ".join(okt.morphs(label.strip())) for label in decoded_labels]

    # Calculate ROUGE-1 score
    rouge = Rouge()
    results = rouge.get_scores(decoded_preds, decoded_labels, avg=True)
    rouge_1_score = results['rouge-1']['f']

    #Calculate Bleu Score
    google_bleu = evaluate.load("google_bleu")
    results = google_bleu.compute(predictions=decoded_preds, references=decoded_labels)
    bleu_score = results['google_bleu']

    #Calculate BERTScore
    bertscore = evaluate.load("bertscore")
    results = bertscore.compute(predictions=decoded_preds, references=decoded_labels, model_type="distilbert-base-uncased")
    bert_score_f1 = sum(results['f1'])/len(predictions)

    # Calculate the mean of the three scores
    mean_score = (rouge_1_score + bleu_score + bert_score_f1) / 3.0

    return mean_score

In [86]:
def preprocess_logits_for_metrics(logits, labels):

    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [19]:
from huggingface_hub import notebook_login

notebook_login()

In [87]:
batch_size = 32
num_train_epochs = 3
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args  = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-koGPT-complete_story",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=False, #구글링 결과는 이걸 True로 하라는데 False 로 하니까 갑자기 해결됐다. 뭐지? 아마도 이걸 True로 해서 list, 즉 결과를 반환해주어버려서 compute_metrics에 list가 들어갔나?
    logging_steps = logging_steps,
    push_to_hub=True,
)

training_args.max_seq_length = 32


In [88]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics if training_args.predict_with_generate else None,
    preprocess_logits_for_metrics = preprocess_logits_for_metrics
)


In [89]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.063000,1.011405
2,1.001300,1.017933
3,0.954600,0.995404


TrainOutput(global_step=11265, training_loss=1.006298741350373, metrics={'train_runtime': 2233.0307, 'train_samples_per_second': 161.404, 'train_steps_per_second': 5.045, 'total_flos': 1.177185927168e+16, 'train_loss': 1.006298741350373, 'epoch': 3.0})

In [93]:
trainer.evaluate()

{'eval_loss': 0.9954036474227905,
 'eval_runtime': 27.6674,
 'eval_samples_per_second': 542.769,
 'eval_steps_per_second': 16.988,
 'epoch': 3.0}

In [90]:
trainer.push_to_hub(commit_message="Training complete", tags="text generation")

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

'https://huggingface.co/Tiabet/kogpt2-base-v2-finetuned-koGPT-complete_story/tree/main/'

In [92]:
predictions,_,_ = trainer.predict(tokenized_datasets['test'])

(array([3, 3, 3, ..., 3, 3, 3]),
 array([[3, 3, 3, ..., 3, 3, 3],
        [3, 3, 3, ..., 3, 3, 3],
        [3, 3, 3, ..., 3, 3, 3],
        ...,
        [3, 3, 3, ..., 3, 3, 3],
        [3, 3, 3, ..., 3, 3, 3],
        [3, 3, 3, ..., 3, 3, 3]]))

In [97]:
predictions[1][2]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [103]:
generated_text = tokenizer.decode(predictions[0], skip_special_tokens=True)
print(generated_text)

이는이는 기 기 기 안내 것을 말다. 안내 낙엽 낙엽 산 낙엽 낙엽 옮겨 붙다. 옮겨벌벌 벌 벌에게냐라고 말 했다.된다고 나를 나를 나를 나를주지 않았다.주지 AT AT AT AT AT AT 은행에 갔다. 뽑 그래서 그래서 그래서 그래서 119 119에 불렀다.고를 남편의 남편의 남편의 남편을백했다.백을 그들은 그들은 밖으로 밖으로 나갔다. 나갔 그 그 그 자전거거를춰갔다.춰 나는이이이웠다.이 대체 학생들은 강의 강의 불만을 못했다. 줄어들 따라 따라 따라 움직다. 따라 세계수는 부담 부담 부담감을 못했다. 부담키 쿠를를 칭찬다고 칭찬었다. 칭찬 언 기억 기억 기억 기억 기억 말해 말했다. 말 그는 그는 그는 그는 그는 일을 일을 일을했다. 일을 나는 나는 나는난 화를 말했다. 말 부모 부모 부모 부모 부모 부모 선물을 선물을하셨다.하 그녀는 그녀는 아들을 아들을 찾아 생각했다. 수 한 한 아이들이 아이들이 책을 책을 책을기 시작다. 앉에게에게냐냐냐고 물었다.고빠빠빠빠빠빠 표를 표를다고 말했다.다고 견인 견인 견인 견인 견인 전화를 요청했다. 전화를우스를우스를우스를보니우스를 색이 색이 발견다.색이 교수 교수 교수 교수님께 털어 구했다. 털어 동 동을 화를쳤다. 들 등급 식 식 식 식당을 요구다.당을 나는 친구 친구 나 평평을평을했다.평을 숙주 숙주 숙주 숙주 숙주 숙주 꺼 먹다.고 그에게 그에게 그에게 그에게 술을 잔 건 말했다. 건 준 준 준 출장근근근 갔다.근 엄마는준 엄 엄 엄마가 씻라고라고 화를셨했다.화를 이탈당에서뉴뉴있게 주문다. 주문 만 만 만 만 꽃 꽃잎 꽃을 꽃을 꺾다. 꺾마는 엄 엄 키우는 키우 키우 키우하 말했다.라고 형 형 형 돈을 돈을 갚했다. 사업이역에서역에서 돈을 나를 놀 놀다. 놀 조카 조카 조카 조카 조카 잘다고 못다.다고 피가 피가 피가 피가렀다. 피가 주전 주전 주전 주전 주전 워다.워 그림을이는 그림을 그림을 그림을 그림을 그림을 그렸했다. 그렸 카드를 카드를 카드를증을 요구했다. 달라고 침 침 프레 프레지 물어봤다. 바꾸 짜

In [104]:
from transformers import pipeline

hub_model_id = "Tiabet/kogpt2-base-v2-finetuned-koGPT-complete_story"
generator = pipeline("text-generation", model=hub_model_id)

In [114]:
generator(dataset['test']['sentence1'][1] + dataset['test']['sentence3'][1], max_length=256)

[{'generated_text': '첫 번째 문장 : 언덕 위에 있는 남성이 팔을 위로 들고 있었다.세 번째 문장 : 그러자 다른 사람이 제발 생쇼하지 말라고 조소했다. 두 번째 문장 : 양 내 친구가'}]

In [106]:
dataset['test']['sentence1'][0]

'첫 번째 문장 : 서영이는 기차에 타서 자리에 앉았다.'